In [1]:
# day11 - https://adventofcode.com/2019/day/11

In [2]:
# this object is where the Intcode program lives and some useful methods
# to fill it, clear it, read from it, write to it
class Computer:
    import logging
    global set_trace
    global inputs
    global outputs
    
    def __init__(self, size, stage, rbase=0):
        # size is array size[0] = program memory, size[1] = general memory
        self.size = size  # must specify the memory size
        self.stage = stage
        # self.storage = [None] * (size[0] + size[1])  # initialize with None
        self.storage = [0] * (size[0] + size[1])  # initialize with 0s
        # initial relative_base
        self.rbase = rbase
        # program base memory is 0
        self.prog_base = 0
        # memory space is immediately above the programming space
        self.mem_base = size[0] + 1
    
    # set the releative_base value
    def set_rbase(self, val):
        self.rbase = val
        return
        
    # dump out the contents between 2 positions (defaults all the memory)
    def dump(self,start=0,end=-1):
        if(end == -1):
            end = self.size
        for i in range(start, end):
            print("{}: {}".format(i,self.storage[i]))

    # fill the memory with a sequence, starting at a certain position - which defaults to 0
    def fill(self, codelist, start_pos = 0):
        i = start_pos
        for j in codelist:
            self.storage[i] = j
            i+=1
    
    # fill (write to) memory with a sequence, allowing direct or reference starting location
    def mode_fill(self, s, mem_location, mode):
        # mem_location: where to begin fill or to reference to begin
        # mode (0 - position, 1 - immediate, 2 - relative)
        if(set_trace): import pdb; pdb.set_trace()
        if(mode == '0' or mode == '2'):
            # these are positional references
            i = self.relative_location(self.read(mem_location), mode)
            for j in s:
                self.storage[i] = j
                i+=1
        elif(mode == '1'):
            # this is the direct reference
            i = mem_location
            for j in s:
                self.storage[i] = j
                i+=1

    # load the program into program space
    def load_program(self, s):
        self.mode_fill(s, self.prog_base, "1")
        return
                
    # get data from input_buffer for this stage
    # if it has 2 values, then it hasn't read in the amp_setting
    # that state will return the last value and remove it

#     def getfrominput(self):
#         if(len(inputs[self.stage].contents) > 1):
#             # pop a value off and return it
#             return(inputs[self.stage].contents.pop())
#         else:
#             # return just that last value but leave it
#             return(inputs[self.stage].contents[0])

    # get data from single input buffer
    def getfrominput(self):
        return(inputs)

    # put data to output_buffer for this stage (which is next stage's input)
    # put this output value into position [0] of that buffer (position [1] will have possible amp_setting)
#     def puttooutput(self, val):
#         logging.info("outputting: {} to stage: {}".format(val, next_stage(self.stage,5)))
#         inputs[next_stage(self.stage,5)].contents[0] = val
#         return

    # output to single output buffer
    def puttooutput(self, val):
        global outputs
        logging.info("before put - outputs: {}".format(outputs))
        outputs.append(val)
        print(outputs)
        logging.info("after put - outputs: {}".format(outputs))
        return

    # read the contents from just one position
    def read(self, pos):
        return self.storage[pos]
   
    # return the value based on parameter mode
    def mode_read(self, pos, mode):
        if(set_trace): import pdb; pdb.set_trace()
        # mode: 0 - position, 1 - immediate, 2 - relative
        if(mode == '0' or mode == '2'):
            return int(self.storage[self.relative_location(self.read(pos), mode)])
        elif(mode == '1'):
            return int(self.read(pos))

    # clear the memory
    def clear(self):
        self.storage = [None] * self.size

    # return the location adjusted by relative_base, based on mode
    def relative_location(self, location_val, mode):
        # location - location value read from prog
        # mode - the parameter mode -'0': position, '2': relative
        logging.info("relative_location() - location_val: {}, mode: {}, self.rbase: {}".format(location_val, mode, self.rbase))
        return((location_val + self.rbase) if int(mode) else location_val)
        
    # this function will execute a single sequence (operator + subsequent operands and output location)
    def exec_seq(self, ctr):
        global set_trace
        # these string representations of opcodes are used in logging
        opcodes = {"00":"NOT-AN-OPCODE","99":"STOP","01":"ADD","02":"MULTIPLY","03":"INPUT","04":"OUTPUT","05":"JUMP-TRUE","06":"JUMP-FALSE","07":"LESS-THAN","08":"EQUALS-TO", "09":"ADJUST-RBASE"}
        # ctr - where to start
        # m - the memory object to work from
        # the first in the sequence is the 'enhnanced opcode' (includes 3 parameter modes in most significant position)
        # zero-pad to 5 - most significant 0 is implied
        enhanced_opcode = str(self.read(ctr)).zfill(5)
        # first 3 characters are the parameter modes
        pmodes = enhanced_opcode[:3]
        # last 2 characters are the opcode
        opcode = enhanced_opcode[3:]
        if(set_trace): import pdb; pdb.set_trace()
        # ===================   99   -  STOP  ============================= 
        if(opcode == '99'):
            prog_status[self.stage] = "STOP"
            logging.info("opcode: {} - {}".format(opcode, opcodes[opcode]))
            return("STOP")
        # ===================   01   -  ADD  ============================= 
        elif(opcode == "01"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(set_trace): import pdb; pdb.set_trace()
            calcval = operand1 + operand2
        # ===================   02   -  MULTIPLY  ============================= 
        elif(opcode == "02"):
            if(set_trace): import pdb; pdb.set_trace()
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            calcval = operand1 * operand2
        # ===================   03   -  INPUT  ============================= 
        elif(opcode == "03"):
            if(set_trace): import pdb; pdb.set_trace()
            # get the input
            # val = getfrominput()
            # the input buffer is the one for this amp stage
            val = self.getfrominput()
            # store it in address provided by next code
            self.mode_fill([val], ctr+1, pmodes[2])
            # return next prog counter
            logging.info("opcode: {}-{}, pmodes: {}, from-input: {}, @ctr+1: {}, ctr in: {}, returning: {}".format(opcode,opcodes[opcode],pmodes,val,self.read(ctr+1),ctr,ctr+2))
            return(ctr+2)
        # ===================   04   -  OUTPUT  ============================= 
        elif(opcode == "04"):
            if(set_trace): import pdb; pdb.set_trace()
            # read the val (based on mode) specified from next code and send to output
            self.puttooutput(self.mode_read(ctr+1, pmodes[2]))
            prog_status[self.stage] = "OUTPUT"
            logging.info("opcode: {}-{}, pmodes: {}, @ctr+1: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),ctr,ctr+2))
            return(ctr+2)
        # ===================   05   -  JUMP-IF-TRUE  ============================= 
        elif(opcode == "05"):
            val_to_test = self.mode_read(ctr+1, pmodes[2])
            if(val_to_test):
                ret_ctr = self.mode_read(ctr+2, pmodes[1])
            else:
                ret_ctr = ctr+3
            logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, val_to_test: {}, ctr+2: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),val_to_test, self.read(ctr+2), self.read(ctr+3), ctr, ret_ctr))
            return(ret_ctr)
        # ===================   06   -  JUMP-IF-FALSE  ============================= 
        elif(opcode == "06"):
            val_to_test = self.mode_read(ctr+1, pmodes[2])
            if(val_to_test):
                ret_ctr = ctr+3
            else:
                ret_ctr = self.mode_read(ctr+2, pmodes[1])
            logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, val_to_test: {}, ctr+2: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),val_to_test, self.read(ctr+2), self.read(ctr+3), ctr, ret_ctr))
            return(ret_ctr)
        # ===================   07   -  LESS-THAN  ============================= 
        elif(opcode == "07"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 < operand2):
                calcval = 1
            #    logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, ctr+2: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1), self.read(ctr+2), self.read(ctr+3), ctr, ret_ctr))
            else:
                calcval = 0
            #    logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, ctr+2: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1), self.read(ctr+2), self.read(ctr+3), ctr, ret_ctr))

        # ===================   08   -  EQUALS-TO  ============================= 
        elif(opcode == "08"):
            operand1 = self.mode_read(ctr+1, pmodes[2])
            operand2 = self.mode_read(ctr+2, pmodes[1])
            if(operand1 == operand2):
                calcval = 1
            else:
                calcval = 0
        # ===================   09   -  ADJUST-RELATIVE-BASE  ============================= 
        elif(opcode == "09"):
             # modify rbase by the value in next position
            self.rbase = self.rbase + self.mode_read(ctr+1, pmodes[2])
            logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, self.rbase: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),self.rbase,ctr,ctr+2))
            return(ctr+2)
        # ===================   ?? - NOT KNOWN  ============================= 
        else:
            prog_status[self.stage] = "BADOPCODE"
            logging.info("opcode: {}-{}".format(opcode,opcodes[opcode]))
            return("BADOPCODE")
        # 3-parameter instructions get here
        # fill the output destination with value depending on mode
        if(set_trace): import pdb; pdb.set_trace()
        self.mode_fill([calcval], ctr+3, pmodes[0])
        logging.info("opcode: {}-{}, pmodes: {}, ctr+1: {}, op1: {}, ctr+2: {}, op2: {}, calcval: {}, ctr+3: {}, ctr in: {}, returning: {}".format(opcode, opcodes[opcode],pmodes,self.read(ctr+1),operand1,self.read(ctr+2),operand2,calcval,self.read(ctr+3),ctr,ctr+4))
        # and return the next postion in the memory
        return(ctr+4)

In [3]:
inputs = None
outputs = [None] * 2
prog_status = [None] * 1

In [4]:
# run parameters / programs
run_name = "day11"

# read the sequence / prog from a file
ifn = "day11-paint-program.txt"
d = open(ifn)
s = d.read()
# make a list of integers from the input comma-separated strings
seq = [int(j) for j in s.split(',')]

In [5]:
inputs = 2
prog_ctr = 0
set_trace = False
set_trace_main = False
ops_max = 300
ops_ctr = 0

In [6]:
# this is used for logging
import time
year, month, day, hour, minute = time.strftime("%Y %m %d %H %M").split()
# log file name
lfn = "{}-{}{}{}{}{}.log".format(run_name,year,month,day,hour,minute)
# set up the logging
import logging
logging.basicConfig(filename=lfn, level=logging.ERROR)
logging.info("==============================================================================")

In [7]:
# initialize computer
# size array is [prog_space, memory_space]
comp = Computer([len(seq),2000], 0)

In [8]:
# load the program
comp.load_program(seq)

In [9]:
class Panel:
    def __init__(self, l, c):
        self.loc = l
        self.c = c
        return

In [10]:
class Robot:
    def __init__(self,l):
        self.loc = l
        return
    def move(self, d):
        self.loc[0] += d[0]
        self.loc[1] += d[1]
        return
        

In [11]:
# this is used to help adjust the bearing
a = [1,3]
# these are 'move left' and 'move right' location incrementers, indexed by directional indicator
d = [[(-1,0), (1,0)], [(0,-1), (0,1)], [(1,0), (-1,0)], [(0,1), (0,-1)]]
# starting bearing is 'up' = 0
b = 0
# panels is an array of Panel()
panels = []
# starting point is on this panel, which is black
panels.append(Panel((0,0),c=0))
r = Robot(panels[0].loc)
# so that is the first input
input = panels[0].c

In [12]:
# run it
while(prog_ctr != "STOP"):
    if(set_trace_main): import pdb; pdb.set_trace()
    logging.info("prog_ctr: {}".format(prog_ctr))
    prog_ctr = comp.exec_seq(prog_ctr)
    if(len(outputs) == 2):
        # process the outputs
        # see if the panel the robot's location is already in the panels array
        i = [i for i,p in enumerate(panels) if p.loc == r.loc]
        if(len(i) == 0):
            # that location not in array, so add it with the color specified by program
            panels.append(r.loc, outputs[0])
        else:
            # that location is in the array, update the color
            panels[i[0]].c = outputs[0]
        # now move in the direction provided by the outputs[1]
        r.move(d[b][outputs[1]])
        # adjust the bearing based on that move
        b = (b + a[outputs[1]]) % 4
        # now set the input based on the color of the current panel
        # see if the location is already in panels array
        # see if the panel the robot's location is already in the panels array
        i = [i for i,p in enumerate(panels) if p.loc == r.loc]
        if(len(i) == 0):
            # that location not in array, so add it with black (default)
            panels.append(r.loc, 0)
            input = 0
            # and note that this panel was a new one
            last_new = 1
        else:
            # that location is in the array, use the color for the input
            input = panels[i[0]].c
            last_new = 0
        # clear the outputs
        outputs = []
    ops_ctr += 1

TypeError: list indices must be integers or slices, not NoneType

In [ ]:
# print the # of panels that were painted at least once
print("the # of panels painted at least once is: {}".format(len(panels) - 1 + last_new)))